In [1]:
!pip install torch torchvision

Defaulting to user installation because normal site-packages is not writeable
distutils: /home/abaxter/.local/lib/python3.9/site-packages
sysconfig: /home/abaxter/.local/lib64/python3.9/site-packages
user = True
home = None
root = None
prefix = None


In [2]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
distutils: /home/abaxter/.local/lib/python3.9/site-packages
sysconfig: /home/abaxter/.local/lib64/python3.9/site-packages
user = True
home = None
root = None
prefix = None


In [3]:
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
distutils: /home/abaxter/.local/lib/python3.9/site-packages
sysconfig: /home/abaxter/.local/lib64/python3.9/site-packages
user = True
home = None
root = None
prefix = None


In [2]:
import torch
import torch.nn as nn
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms.functional import to_pil_image
from PIL import Image
import os
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
class CSVDataset(Dataset):
    def __init__(self, dataframe, image_root_dir, target_columns=None, transform=None,
                 save_dir=None, use_saved_images=False):
        self.data = dataframe
        self.image_root_dir = image_root_dir
        self.target_columns = target_columns
        self.transform = transform
        self.save_dir = save_dir
        self.use_saved_images = use_saved_images

        if self.save_dir:
            os.makedirs(self.save_dir, exist_ok=True)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        
        # Use index for the saved tensor filename
        #saved_image_path = os.path.join(self.save_dir, f"{idx}.pt")
        image_index = row['Unnamed: 0']
        saved_image_path = os.path.join(self.save_dir, f"{image_index}.pt")



        if self.use_saved_images:
            if os.path.exists(saved_image_path):
                image_tensor = torch.load(saved_image_path)
            else:
                raise FileNotFoundError(f"Saved tensor not found: {saved_image_path}")
        else:
            original_image_path = os.path.join(self.image_root_dir, row['Path'])
            image = Image.open(original_image_path).convert("L")
            image_tensor = self.transform(image) if self.transform else transforms.ToTensor()(image)

            if self.save_dir:
                torch.save(image_tensor, saved_image_path)

        if self.target_columns:
            labels = pd.to_numeric(row[self.target_columns], errors='coerce').fillna(0).astype(float).values
            labels = torch.tensor(labels, dtype=torch.float32)
            return image_tensor, labels

        return image_tensor

In [5]:
class MultiLabelDenseNet121(nn.Module):
    def __init__(self, num_classes):
        super(MultiLabelDenseNet121, self).__init__()

        # Load pre-trained DenseNet-121
        self.base_model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        
        # Replace the classifier with a custom head
        self.base_model.classifier = nn.Sequential(
            nn.Linear(self.base_model.classifier.in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
            nn.Tanh()  # Output values in [-1, 1] for each class
        )

    def forward(self, x):
        return self.base_model(x)

In [6]:
image_root = '/central/groups/CS156b/2025/CodeMonkeys/input_images'
image_root_dir = "input_images/train"
train_save_dir = os.path.join(image_root, 'train')
#test_save_dir = os.path.join(image_root, 'test')
full_train_df = pd.read_csv('train2023.csv')
#filtered_train_df = full_train_df.iloc[:29692]
filtered_train_df = full_train_df.iloc[:10000]

filtered_train_df = filtered_train_df.dropna(subset=['Pleural Effusion'])  # Drop NaN values
filtered_train_df = filtered_train_df[filtered_train_df['Pleural Effusion'] != 0]  # Drop rows with 'Pleural_Effusion' == 0



In [7]:
label_counts = filtered_train_df['Pleural Effusion'].value_counts()
print(label_counts)


Pleural Effusion
 1.0    3602
-1.0    1903
Name: count, dtype: int64


In [28]:
from sklearn.model_selection import train_test_split

# Define your target columns once
target_columns = ["Pleural Effusion"]

# Step 1: Split the dataframe
train_df, val_df = train_test_split(filtered_train_df, test_size=0.15, random_state=42)

# Step 2: Create training dataset
train_dataset = CSVDataset(
    dataframe=train_df, 
    image_root_dir=image_root, 
    target_columns=target_columns, 
    save_dir=train_save_dir, 
    use_saved_images=True
)

# Step 3: Create validation dataset
val_dataset = CSVDataset(
    dataframe=val_df, 
    image_root_dir=image_root, 
    target_columns=target_columns, 
    save_dir=train_save_dir, 
    use_saved_images=True
)

# Step 4: Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [1]:
def freeze_base_layers(model):
    """
    Freeze initial layers of DenseNet-121 up to denseblock3.
    """
    freeze = True
    for name, child in model.base_model.features.named_children():
        if name == 'denseblock1':
            freeze = False
        if freeze:
            for param in child.parameters():
                param.requires_grad = False
    return model


In [30]:
class_weights = {}

# Loop over each target column
for col in target_columns:
    # Count the occurrences of each class in the column
    counts = filtered_train_df[col].value_counts()
    total = len(filtered_train_df[col])
    
    # Calculate class weights using inverse frequency (you can also experiment with other strategies)
    weights = {
        -1: total / (counts.get(-1, 0) + 1),  # Add 1 to avoid division by zero
        0: total / (counts.get(0, 0) + 1),
        1: total / (counts.get(1, 0) + 1)
    }
    
    # Store weights for each class
    class_weights[col] = weights

# Example: Print out the weights for each class
for col in target_columns:
    print(f"Class weights for {col}: {class_weights[col]}")


Class weights for Pleural Effusion: {-1: np.float64(2.891281512605042), 0: 5505.0, 1: np.float64(1.52789342214821)}


In [31]:
criterion = nn.MSELoss(reduction='none')

def masked_MSE_loss(output, target, class_weights):
    # Create a mask for non-NaN target values
    mask = ~torch.isnan(target)
    
    # Apply the MSE loss
    loss = criterion(output, target)
    
    # Loop through each class and apply the class weights
    for class_idx, col in enumerate(target_columns):
        # Get the class values for the current class
        class_values = target[:, class_idx]
        
        # Apply the class weights to each class value
        weight = torch.tensor([class_weights[col].get(x.item(), 1) for x in class_values], dtype=torch.float32, device=output.device)
        
        # Apply the weight to the loss (broadcast the weight to match the loss shape)
        loss = loss * mask  # Apply mask to exclude NaN targets
        loss[:, class_idx] *= weight  # Apply weight per class
    
    # Return mean loss for valid entries
    return loss.sum() / mask.sum()


In [33]:
import torch.optim as optim

# Hyperparameters and model setup
num_classes = 1  # As you're predicting a single label ('Pleural Effusion')
model = MultiLabelDenseNet121(num_classes=num_classes).to(device)
model = freeze_base_layers(model)  # Freeze layers

# Loss function and optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Scheduler (optional)
#scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()  # Zero the gradients
        
        # Forward pass
        outputs = model(images)
        
        # Compute loss
        loss = masked_MSE_loss(outputs, labels, class_weights)
        running_loss += loss.item()
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Calculate predictions and accuracy
        predicted_class = torch.where(outputs > 0.5, torch.tensor(1.0), 
                                      torch.where(outputs < -0.5, torch.tensor(-1.0), torch.tensor(0.0)))
        correct += (predicted_class == labels).sum().item()
        total += labels.numel()

    avg_loss = running_loss / len(train_loader)
    accuracy = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")
    
    # Evaluate on the validation set
    model.eval()  # Set to evaluation mode
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = masked_MSE_loss(outputs, labels, class_weights)
            val_loss += loss.item()

            # Collect predictions and true labels
            predicted_class = torch.where(outputs > 0, torch.tensor(1.0).to(device), torch.tensor(-1.0).to(device))
            
            all_preds.append(predicted_class.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

            val_correct += (predicted_class == labels).sum().item()
            val_total += labels.numel()

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = val_correct / val_total

    # Flatten the lists for precision, recall, and F1 score calculation
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Compute precision, recall, and F1 for multi-label classification
    val_precision = precision_score(all_labels, all_preds, average='macro', zero_division=1)
    val_recall = recall_score(all_labels, all_preds, average='macro', zero_division=1)
    val_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=1)

    print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    print(f"Validation Precision: {val_precision:.4f}, Validation Recall: {val_recall:.4f}, Validation F1: {val_f1:.4f}")
    
    # Optionally, save the model
    if (epoch+1) % 10 == 0:
        torch.save(model.state_dict(), f"models/pe_dn_epoch_{epoch+1}.pth")
    
    # Step the scheduler (optional)
    #scheduler.step(avg_val_loss)

Epoch [1/10], Loss: 0.8935, Accuracy: 0.7226
Validation Loss: 0.7382, Validation Accuracy: 0.8547
Validation Precision: 0.8423, Validation Recall: 0.8799, Validation F1: 0.8480
Epoch [2/10], Loss: 0.5868, Accuracy: 0.8448
Validation Loss: 0.8181, Validation Accuracy: 0.8753
Validation Precision: 0.8606, Validation Recall: 0.8599, Validation F1: 0.8603
Epoch [3/10], Loss: 0.4430, Accuracy: 0.8820
Validation Loss: 0.9135, Validation Accuracy: 0.8402
Validation Precision: 0.8282, Validation Recall: 0.8645, Validation F1: 0.8330
Epoch [4/10], Loss: 0.3506, Accuracy: 0.9081
Validation Loss: 0.7771, Validation Accuracy: 0.8705
Validation Precision: 0.8523, Validation Recall: 0.8811, Validation F1: 0.8614
Epoch [5/10], Loss: 0.2656, Accuracy: 0.9348
Validation Loss: 0.8102, Validation Accuracy: 0.8644
Validation Precision: 0.8466, Validation Recall: 0.8765, Validation F1: 0.8554
Epoch [6/10], Loss: 0.3074, Accuracy: 0.9149
Validation Loss: 0.7057, Validation Accuracy: 0.8789
Validation Precis

## Testing Model on Validation Set

In [47]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import numpy as np
import torch

def evaluate_model_per_class(val_loader, device, model_path):
    model = MultiLabelResNet50(num_classes=1).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    all_preds = []
    all_labels = []
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = masked_MSE_loss(outputs, labels, class_weights)
            val_loss += loss.item()

            predicted_class = torch.where(outputs > 0, torch.tensor(1.0).to(device), torch.tensor(-1.0).to(device))

            all_preds.append(predicted_class.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

            val_correct += (predicted_class == labels).sum().item()
            val_total += labels.numel()

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = val_correct / val_total

    # Flatten for sklearn metrics
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    val_precision = precision_score(all_labels, all_preds, average='macro', zero_division=1)
    val_recall = recall_score(all_labels, all_preds, average='macro', zero_division=1)
    val_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=1)

    print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    print(f"Validation Precision: {val_precision:.4f}, Validation Recall: {val_recall:.4f}, Validation F1: {val_f1:.4f}")


# Example usage
evaluate_model_per_class(val_loader, device, "models/model_pe_epoch_10.pth")


Validation Loss: 0.7026, Validation Accuracy: 0.8901
Validation Precision: 0.8748, Validation Recall: 0.8962, Validation F1: 0.8827


In [ ]:
import torch
from collections import Counter
import numpy as np

def print_label_distribution(val_loader):
    all_labels = []

    for _, labels in val_loader:
        all_labels.append(labels)

    all_labels = torch.cat(all_labels, dim=0).cpu().numpy()  # shape: (num_samples, num_classes)

    num_classes = all_labels.shape[1]

    print("Label distribution per class:")
    for i in range(num_classes):
        unique, counts = np.unique(all_labels[:, i], return_counts=True)
        dist = dict(zip(unique, counts))
        print(f"Class {i}: {dist}")
print_label_distribution(val_loader)

Label distribution per class:
Class 0: {np.float32(-1.0): np.int64(1829), np.float32(0.0): np.int64(169), np.float32(1.0): np.int64(252)}
Class 1: {np.float32(-1.0): np.int64(258), np.float32(0.0): np.int64(1891), np.float32(1.0): np.int64(101)}
Class 2: {np.float32(-1.0): np.int64(192), np.float32(0.0): np.int64(1811), np.float32(1.0): np.int64(247)}
Class 3: {np.float32(-1.0): np.int64(56), np.float32(0.0): np.int64(1188), np.float32(1.0): np.int64(1006)}
Class 4: {np.float32(-1.0): np.int64(34), np.float32(0.0): np.int64(2190), np.float32(1.0): np.int64(26)}
Class 5: {np.float32(-1.0): np.int64(409), np.float32(0.0): np.int64(1045), np.float32(1.0): np.int64(796)}
Class 6: {np.float32(-1.0): np.int64(3), np.float32(0.0): np.int64(2199), np.float32(1.0): np.int64(48)}
Class 7: {np.float32(-1.0): np.int64(47), np.float32(0.0): np.int64(2113), np.float32(1.0): np.int64(90)}
Class 8: {np.float32(-1.0): np.int64(50), np.float32(0.0): np.int64(1164), np.float32(1.0): np.int64(1036)}


In [ ]:
# Create dataset
test_dataset = CSVDataset(
    dataframe=df_first10rows_test, 
    image_root_dir=image_root, 
    target_columns=None, 
    transform=image_transforms,  # Pass the transform
    save_dir=test_save_dir, 
    use_saved_images=False  # Set to True if you want to load tensors from CSV
)

# Create DataLoader
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# Iterate through batches
for batch_idx, (images) in enumerate(test_loader):
    print(f"Batch {batch_idx + 1}")
    print("Images shape:", images.shape)

## Run Model

In [48]:
model = MultiLabelResNet50(num_classes=1).to(device)
model.load_state_dict(torch.load('models/model_pe_epoch_10.pth'))
model.eval()  

MultiLabelResNet50(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequenti

In [49]:
average_values = {
    "No Finding": -0.734655,
    "Enlarged Cardiomediastinum": -0.275805,
    "Cardiomegaly": 0.190770,
    "Lung Opacity": 0.836288,
    "Pneumonia": 0.031183,
    #"Pleural Effusion": 0.384547,
    "Pleural Other": 0.521795,
    "Fracture": 0.392374,
    "Support Devices": 0.888289
}

In [57]:
import torch
import os
import pandas as pd
from tqdm import tqdm

# Directory containing the test images
test_dir = 'input_images/test'

# Columns for the prediction output
columns = ["Id", "No Finding", "Enlarged Cardiomediastinum", "Cardiomegaly", "Lung Opacity", 
           "Pneumonia", "Pleural Effusion", "Pleural Other", "Fracture", "Support Devices"]

# Average values for columns other than Pleural Effusion
average_values = {
    "No Finding": -0.734655,
    "Enlarged Cardiomediastinum": -0.275805,
    "Cardiomegaly": 0.190770,
    "Lung Opacity": 0.836288,
    "Pneumonia": 0.031183,
    "Pleural Other": 0.521795,
    "Fracture": 0.392374,
    "Support Devices": 0.888289
}

# Batch size for processing
batch_size = 64  # Adjust based on your GPU's memory
batch = []
batch_filenames = []
predictions = []

# List of all files in the test directory
file_list = [f for f in os.listdir(test_dir) if f.endswith(".pt")]

# Loop through all the files in the test directory
for filename in tqdm(file_list):
    image_path = os.path.join(test_dir, filename)
    image_tensor = torch.load(image_path).to(device)  # Load the image tensor onto the device
    batch.append(image_tensor)
    batch_filenames.append(filename.split('.')[0])  # Store the filename without extension

    # If the batch is full or it's the last file, process the batch
    if len(batch) == batch_size or filename == file_list[-1]:
        input_batch = torch.stack(batch)  # Stack the images into a single tensor
        with torch.no_grad():
            output = model(input_batch).cpu().numpy()  # Make the prediction
        
        # Process each output in the batch
        for i in range(len(output)):
            row = [batch_filenames[i]] + [
                output[i][0] if col == "Pleural Effusion" else average_values.get(col, 0)
                for col in columns[1:]
            ]
            predictions.append(row)  # Add the row to the predictions list
        
        # Reset the batch and filenames for the next batch
        batch = []
        batch_filenames = []

# Convert the predictions to a DataFrame
df_predictions = pd.DataFrame(predictions, columns=columns)

# Save the predictions to a CSV file
df_predictions.to_csv('test_predictions.csv', index=False)

print("Predictions saved to 'test_predictions.csv'")


  0%|          | 64/22596 [00:00<00:58, 385.21it/s]

100%|██████████| 22596/22596 [09:08<00:00, 41.17it/s]


Predictions saved to 'test_predictions.csv'


In [58]:
import os
print(os.getcwd())


/central/groups/CS156b/2025/CodeMonkeys
